# iDASH Tumor classification# iDASH2020 tumor classification 데이터 분석 모델

본 문서에서는 'tumor classficiation' 모델 구동을 설명한다. 

## 개요
Tumor 데이이터 분석은 세 단계로 나누어진다. 
1. tumor 데이터 전처리
2. Scikit-learn 모델 훈련
3. FHE neural RF 모델 훈련 및 활용

## Tumor data
Tumor 데이터는 상당히 비균질한 특성을 가지므로 다양한 전처리 기술이 필요하다.  
[Hong et al. 2021](https://doi.org/10.21203/rs.3.rs-584746/v1)연구에서는 모델의 정확도를 향상시키기 위해 유사한 gene을 그룹화하고, 영향이 적은 gene mutation을 걸러내는 등 도메인 지식에 기반한 다양한 전처리 작업을 수행하였다.  
이 과제는 FHE 활용 RF 모델을 만드는 것에 초점을 두므로 고도의 전처리는 포함하지 않는다. FHE 모델은 Scikit-learn으로 훈련된 최종 모델을 참조하므로 추후에 전처리 단계를 고도화하여 모델의 정확도를 향상시킬 수 있다. 

## 전처리
데이터 전처리 과정은 
1. 산재하는 파일을 읽어서 사용하기 편한 형태로 변환하고,
2. 불필요한 데이터를 정재하는 단계,
3. 그리고 데이터 범위를 표준화하는 단계로 구성된다.

iDASH2020 데이터셋의 기본 구조를 가정하여 Data loader가 작성되어있으며, 작동시에는 데이터 셋의 root 디렉토리 위치만 지정해주면 된다. 


#### 필요한 class를 import

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

from fase import seal
from fase.RF.sktree import TumorRF

### TumorRF instance에 데이터의 root 디렉토리를 설정하여 데이터 전처리 수행

In [2]:
data_dir = "/home/hoseung/Dropbox/DeepInsight/2021ETRI/RF_Tumor_Classification/data/"

# Init TumorRf class
# Pre-defined preprocessing steps are taken automatically. 
tumor_rf = TumorRF(data_dir)

Preprocessing done and train data are ready


### train - validation set 

모델을 훈련하고, 사용된 데이터셋을 pickle로 저장. 

In [3]:
# Split the data and keep validation data separately. 
X_train, X_valid, Y_train, Y_valid = tumor_rf.split_data(fn_dump="tumor_")

# Train the model
tumor_rf.train(X_train, Y_train, fn_out="trained_RF.pickle", ntree=21, depth=8)

# Test the model against validation dataset.
y_preds = tumor_rf.predict(X_valid)

# Caculate validation performance 
tumor_rf.accuracy(Y_valid, y_preds)

Saved Training set as tumor_train.pickle
Saved Validation set as tumor_valid.pickle
Training a Random Forest with 21 trees of depth 8.
Train Done: train accuracy = 57.761%
Test accuracy 57.737%


Depth = 8일 때, 약 60%의 정확도를 얻었음. 

## Data loading

In [8]:
from fase.RF.sktree import extract_sk_dt
RF = [extract_sk_dt(dt) for dt in tumor_rf.rf.estimators_]
pickle.dump(RF, open("trained_rf_ver1", "wb"))

In [21]:
model_dir = "/home/hoseung/Work/data/trained_RF/"

trained_rf = pickle.load(open(model_dir+"whole_rf.pickle", 'rb'))
dd = pickle.load(open(model_dir+"validset.pickle", 'rb'))

X_valid = dd["valid_x"]
Y_valid = dd["valid_y"]
le = dd["label_encoder"]

assert (X_valid.min()>= -1) and (X_valid.max()) <= 1

In [3]:
from cryptotree.tree import NeuralRF#, SigmoidTreeMaker, TanhTreeMaker
from cryptotree.polynomials import plot_graph_function_approximation

max_depth = 8

dilatation_factor = 10
polynomial_degree = dilatation_factor

#plot_graph_function_approximation(torch.tanh,
#                                  dilatation_factor=dilatation_factor,
#                                  polynomial_degree=polynomial_degree)

In [4]:
from cryptotree.tree import NeuralTreeMaker
from sklearn.tree import BaseDecisionTree
from functools import partial
from cryptotree.tree import *
from cryptotree.mine import *

my_tm_tanh = NeuralTreeMaker(torch.tanh, 
                       use_polynomial=True,
                                  dilatation_factor=dilatation_factor, polynomial_degree=polynomial_degree)

In [5]:
#tree_maker = tanh_tree_maker

model =NeuralRF(trained_rf.rf.estimators_, tree_maker=my_tm_tanh)
model.to("cpu")

NeuralRF()

### Without finetuning

각각의 estimator는 decision tree임. 여기서 
tree.tree_.node_count
tree.tree_.children_left
tree.tree_.children_right
tree.tree_.feature
tree.tree_.threshold
tree.n_features_ 추출해냄. 



Linear layer. weight = one-hot code of feature at each node

questions in all branching nodes 

Weight  
[100000] - node 1 asks about feature 1  
[010000] - node 2 asks about feature 2  
[010000] - node 3 asks about feature 1  
[000001] - node 4 asks about feature 6 ..  


Bias   = threshold   
[-0.4,  
 0.4,  
 0.6,  
 0.9,  
 0.2,  
 0.1,  
 .  
 .  
 ...]


# matcher가 하는 일은..?   - 모든 branch node와 모든 leaf node를 연결

예)  
10번 leaf가  0 - 6 - 8 - 10 순으로 도착한다면 layer 1의 0,6,8번 정보를 합쳐서 layer2 -> prediction을 수행함.  


NRF의 아이디어를 정리하자면,
1) 모든 branch 노드를 weight과 bias로 표현  
2) leaf마다 도달하는데 거치는 branch 노드들의 계산 결과의 조합으로 최종 결정을 내림.  
그러니까, branch leaf에 도달하는 순서는 상관하지 않겠다는 의미. -- 왜냐면 yes면 1이고 no면 0이니까 순서랑 무관함.   

그렇다면 layer를 더 쌓는다는게 의미가 있나? 말이 안 되는 건가..?  class 수가 많아지면 어떻게 하지..  


한편,  
NRF는 sklearn에서 train한 RF를 NN으로 옮기는 일. 그냥 처음부터 Linear 3개짜리 NN을 만들면 안 되나??  
hyperparameter를 sklearn에서 결정해준다는 점은 의미가 있음.  

crytotree에서는 서로 다른 크기/깊이의 tree를 가장 큰 tree에 맞춰 pad함.  (tree.py: L220)
왜..? 

### Sigmoid 트리는 성능이 매우 떨어짐. tanh는 얼추 비슷

In [9]:
with torch.no_grad():
    #sigmoid_neural_pred = sigmoid_neural_rf(torch.tensor(X_train_normalized).float()).argmax(dim=1).numpy()
    #tanh_neural_pred = tanh_neural_rf(torch.tensor(X_train_normalized).float()).argmax(dim=1).numpy()
    tanh_neural_pred = model(torch.tensor(X_valid[::100]).float()).argmax(dim=1).numpy()
    
pred = trained_rf.predict(X_valid[::100])
print(f"Original accuracy : {(pred[::100] == Y_valid[::100]).mean()}")

#print(f"Accuracy of sigmoid  : {(sigmoid_neural_pred == y_train).mean()}")
print(f"Accuracy of tanh : {(tanh_neural_pred[::100] == Y_valid[::100]).mean()}")

#print(f"Match between sigmoid and original : {(sigmoid_neural_pred == pred).mean()}")
print(f"Match between tanh and original : {(tanh_neural_pred[::100] == pred[::100]).mean()}")

/tmp/ipykernel_150923/585595069.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print(f"Original accuracy : {(pred[::100] == Y_valid[::100]).mean()}")


AttributeError: 'bool' object has no attribute 'mean'

Original accuracy : 0.5801656288368316
Accuracy of tanh : 0.3580733051904536
Match between tanh and original : 0.48896215385969977

# NRF도 FHE RF처럼 성능이 확 떨어짐...(1) tanh approx.가 문제인가 아니면 (2) class가 많은게 문제인가, 아니면 (3) Unbalanced class / 부족한 preprocessing이 문제인가? 

polynomial degree를 10에서 16으로 올려도 정확히 똑같은 결과. 

fine-tune하면 좋아짐.

### With finetuning

Here we first define our Pytorch Dataset.

In [10]:
from torch.utils import data
import numpy as np

class TabularDataset(data.Dataset):
    """minimum dataset class to convert a ndarray into a torch tensor"""
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X, self.y = X,y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Load data and get label
        X = torch.tensor(self.X[index]).float()
        y = torch.tensor(self.y[index])

        return X, y

Then we split our training data into training and validation.

In [11]:
dd = pickle.load(open(model_dir+"trainset.pickle", 'rb'))

X_train = dd["train_x"]
Y_train = dd["train_y"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.20, random_state=seed)
X_train_normalized, X_valid_normalized, y_train, y_valid = train_test_split(pipe.transform(X_train), 
                                                                            y_train,
                                                                            train_size=0.8)

Then we create the Pytorch dataloaders.

In [12]:
train_ds = TabularDataset(X_train, Y_train)
valid_ds = TabularDataset(X_valid, Y_valid)

bs = 128

train_dl = data.DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = data.DataLoader(valid_ds, batch_size=bs)
fix_dl = data.DataLoader(train_ds, batch_size=bs, shuffle=False)

Here we will just define the model, which is a sigmoid Neural Random Forest.

Because we only want to train the last layer, we will freeze the first two layers and check they are frozen.

In [15]:
trained_rf.rf.estimators_[0]

DecisionTreeClassifier(max_depth=8, max_features='auto',
                       random_state=1150192196)

In [13]:
model.freeze_layer("comparator")
model.freeze_layer("matcher")

for p in model.parameters():
    print(p.shape, p.requires_grad)

torch.Size([7, 248, 21]) False
torch.Size([248, 21]) False
torch.Size([249, 248, 21]) False
torch.Size([249, 21]) False
torch.Size([11, 249, 21]) True
torch.Size([11, 21]) True


# 메모리 부족 


NeuralRF.forward()에서 모든 계산을 Einsum으로 하는데,  
Einsum으로 계산하면 전체 input을 한번에 계산함. 16만 개를 한번에 하기는 좀 무리. 

In [ ]:
pred = trained_rf.predict(X_train)

with torch.no_grad():
    neural_pred = model(torch.tensor(X_train[:10000]).float()).argmax(dim=1).numpy()

print(f"Original accuracy : {(pred == Y_train).mean()}")
print(f"Accuracy : {(neural_pred == Y_train[:10000]).mean()}")
print(f"Same output : {(neural_pred == pred[:10000]).mean()}")

## Pytorch fine tune



In [16]:
import time 
import copy 

def train_model(model, 
                dataloader, 
                criterion, 
                optimizer, 
                scheduler, 
                num_epochs=25, 
                phase="train"):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            #with torch.set_grad_enabled(phase == 'train'):
            #print(model.comparator.requires_grad)
            #print(model.matcher.requires_grad)
            #print(model.head.requires_grad)
        
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / len(dataloader)
        epoch_acc = running_corrects.double() / len(dataloader)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))
        print("updating..?", model.head[0].min(), model.head[0].max())
        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model

In [32]:
from torch.optim import lr_scheduler
device = "cpu"

#criterion = nn.CrossEntropyLoss()
criterion = CrossEntropyLabelSmoothing()

lr = 5e-3

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

model.train()

print("updating..?", model.head[0].min(), model.head[0].max())
        
model = train_model(model, train_dl, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=2, phase="train")

updating..? tensor(-0.1759, grad_fn=<MinBackward1>) tensor(0.4055, grad_fn=<MaxBackward1>)
Epoch 0/1
----------
train Loss: 233.4254 Acc: 58.5607
updating..? tensor(-0.3014, grad_fn=<MinBackward1>) tensor(0.6741, grad_fn=<MaxBackward1>)
Epoch 1/1
----------
train Loss: 228.4141 Acc: 62.9072
updating..? tensor(-0.4140, grad_fn=<MinBackward1>) tensor(0.9144, grad_fn=<MaxBackward1>)
Training complete in 1m 26s
Best val Acc: 0.000000


Fine tune 되나? Fastai랑 다른거 없는 것 같은데... 

In [33]:
print("updating..?", model.head[0].min(), model.head[0].max())

updating..? tensor(-0.4140, grad_fn=<MinBackward1>) tensor(0.9144, grad_fn=<MaxBackward1>)


In [44]:
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
num_epochs=2
dataloader = train_dl
model.train()
phase="train"
scheduler=exp_lr_scheduler

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        #with torch.set_grad_enabled(phase == 'train'):
        outputs = model(inputs)
        #print(model.comparator.requires_grad)
        #print(model.matcher.requires_grad)
        #print(model.head.requires_grad)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize only if in training phase
        if phase == 'train':
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    if phase == 'train':
        scheduler.step()

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = running_corrects.double() / len(dataloader)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))

    # deep copy the model
    if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())


time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
model.load_state_dict(best_model_wts)


Epoch 0/1
----------
train Loss: 237.3078 Acc: 55.5949
Epoch 1/1
----------
train Loss: 237.3078 Acc: 55.5949
Training complete in 1m 36s
Best val Acc: 0.000000


<All keys matched successfully>

----------
train Loss: 208.2449 Acc: 66.2010
Epoch 23/24

Now we can define our fastai Learner, with the dataset, the model, and the loss function, which is a Label Smoothing Cross Entropy here.

# Fastai 대체하기 

너무 티나니까..

Learner(data, model, loss_fuction, metrics)
1. data:
    pytorch dataloader과 똑같은 기능. Image와 label외에 bbox나 segmap등도 지원함.
    
2. model
    torch 모델
   
3. loss_function

import fastai
fastai.__version__
# Need Fastai 1, NOT 2

from fastai.basic_data import DataBunch
from fastai.tabular.learner import Learner
from fastai.metrics import accuracy

from cryptotree.tree import CrossEntropyLabelSmoothing
import torch.nn as nn

data = DataBunch(train_dl, valid_dl,fix_dl=fix_dl)

criterion = CrossEntropyLabelSmoothing()

learn = Learner(data, model, loss_func=criterion, metrics=accuracy)

We will use fastai lr finder to have an idea of what learning rate to choose.

learn.lr_find(num_it=500)
learn.recorder.plot()

Here we can see that a good learning rate should be around 1e-1.

We can now fine tune our model.

learn.fit_one_cycle(5,1e-2 / 2)

Here we can have a look at the performance of the Neural Random Forest tuned with respect to the original sklearn Random Forest.

In [34]:
sl = slice(None, None, 5)

pred = trained_rf.predict(X_valid[sl])

with torch.no_grad():
    neural_pred = model(torch.tensor(X_valid[sl]).float()).argmax(dim=1).numpy()

print(f"Original accuracy : {(pred == Y_valid[sl]).mean()}")
print(f"Accuracy : {(neural_pred == Y_valid[sl]).mean()}")
print(f"Same output : {(neural_pred == pred).mean()}")

Original accuracy : 0.5663046934233374
Accuracy : 0.5084012033164576
Same output : 0.8140362462396361


#### Fastai 결과  

Original accuracy : 0.5663046934233374  
Accuracy : 0.5171325849291951  
Same output : 0.7578325629173087

In [35]:
pickle.dump(model, open("fine_tuned_torch.pickle","wb"))

## fine tune으로 성능이 썩 좋아짐 :)

## Homomorphic Random Forest

In [17]:
import numpy as np 

dilatation_factor = 16
degree = dilatation_factor

PRECISION_BITS = 28
UPPER_BITS = 9

polynomial_multiplications = int(np.ceil(np.log2(degree))) + 1
n_polynomials = 2
matrix_multiplications = 3

depth = matrix_multiplications + polynomial_multiplications * n_polynomials

poly_modulus_degree = 16384 # 2**14

moduli = [PRECISION_BITS + UPPER_BITS] + (depth) * [PRECISION_BITS] + [PRECISION_BITS + UPPER_BITS]
print(moduli)
print(sum(moduli))

[37, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 37]
438


In [18]:
sl = slice(None, None, 1000)


pred = trained_rf.predict(X_valid[sl])

with torch.no_grad():
    neural_pred = model(torch.tensor(X_valid[sl]).float()).argmax(dim=1).numpy()

print(f"Original accuracy : {(pred == Y_valid[sl]).mean()}")
print(f"Accuracy : {(neural_pred == Y_valid[sl]).mean()}")
print(f"Same output : {(neural_pred == pred).mean()}")

Original accuracy : 0.5427872860635696
Accuracy : 0.3643031784841076
Same output : 0.4841075794621027


In [41]:
from cryptotree.seal_helper import create_seal_globals, append_globals_to_builtins
import builtins

create_seal_globals(globals(), poly_modulus_degree, moduli, PRECISION_BITS, use_symmetric_key=False)
append_globals_to_builtins(globals(), builtins)

We can then create our Homomorphic Random Forest from the Neural Random Forest we fine tuned earlier.

We first have to extract the weights using the Homomorphic Random Forest class, and pass it to the Homomorphic Tree Evaluator which will do the computation, using the polynomial activation given, and the SEAL context.

A featurizer is also created for the client side, in order to preprocess, encode and encrypt the data.

In [45]:
b0, w1, b1, w2, b2 = h_rf.return_weights()

In [48]:
nexample = len(b0)
print(nexample, nexample / ntrees)

10437

# Weight과 bias를 encode 해야하는데, 하나의 ciphertext에 다 안 들어감. 
# 1. Ciphertext를 키운다 -> 2**14
# 2. 트리를 각각 계산한다. (지금은 21개를 하나로 합쳐서 10437개가 됨) -- 이게 더 맞는 말인 듯.

In [19]:
from cryptotree.cryptotree import HomomorphicNeuralRandomForest, HomomorphicTreeEvaluator, HomomorphicTreeFeaturizer
from cryptotree.polynomials import polyeval_tree

h_rf = HomomorphicNeuralRandomForest(model)

tree_maker = my_tm_tanh
tree_evaluator = HomomorphicTreeEvaluator.from_model(h_rf, 
                                                     tree_maker.coeffs, 
                                                     polyeval_tree, 
                                                     evaluator, 
                                                     encoder, 
                                                     relin_keys, 
                                                     galois_keys, 
                                                     scale)

homomorphic_featurizer = HomomorphicTreeFeaturizer(h_rf.return_comparator(), encoder, encryptor, scale)

NameError: name 'evaluator' is not defined

Now we can take some data, encrypt it, and pass it to our Homorphic Tree Evaluator : 

In [45]:
i = 0

x = X_train_normalized[i]
ctx = homomorphic_featurizer.encrypt(x)

outputs = tree_evaluator(ctx)

We can decrypt and decode the data to see the output :

In [46]:
ptx = seal.Plaintext()
decryptor.decrypt(outputs, ptx)

homomorphic_pred = encoder.decode(ptx)[:2]

We can see what was the original output, if we had used only the Neural Random Forest : 

In [47]:
x = X_train_normalized[i]

pred = rf.predict_proba(x.reshape(1,-1))
neural_pred = model(torch.tensor(x).float().unsqueeze(0))

print(f"Original Random Forest output : {pred}")
print(f"Neural Random Forest output : {neural_pred.detach()}")
print(f"Homomorphic Random Forest output : {homomorphic_pred}")

Original Random Forest output : [[0.604789 0.395211]]
Neural Random Forest output : tensor([[0.1420, 0.1236]])
Homomorphic Random Forest output : [0.256069 0.002739]


Here we can see our Neural Random Forest, and Homomorphic Random Forest have very similar outputs.

## Comparison between models

We will now compare a linear model, a RF, a NRF and a HRF. 

Because computation are done one at a time with HRF, we will use Fastai `parallel` to speedup inference taking advantage of the multiple CPUs.

In [48]:
from fastai.core import parallel
import multiprocessing

def predict(x, index):
    """Performs HRF prediction"""
    
    # We first encrypt and evaluate our model on it
    ctx = homomorphic_featurizer.encrypt(x)
    outputs = tree_evaluator(ctx)
    
    # We then decrypt it and get the first 2 values which are the classes scores
    # ptx = seal.Plaintext()
    ptx = decryptor.decrypt(outputs)
    
    homomorphic_pred = encoder.decode(ptx)[:2]
    homomorphic_pred = np.argmax(homomorphic_pred)
    
    return index, homomorphic_pred

# We get the number of cores
cores = multiprocessing.cpu_count()

# We compute the outputs
hrf_pred = parallel(predict, X_valid_normalized[:320,:], max_workers=cores)

# Because the outputs are unordered we must first sort by index then take the predictions
hrf_pred = np.array(sorted(hrf_pred, key = lambda x:x[0]))[:,1]

We compute the predictions of the NRF.

In [49]:
outputs = []

for x,y in valid_dl:
    with torch.no_grad():
        pred = model(x)
    outputs.append(pred)
    
nrf_pred = torch.cat(outputs).argmax(dim=1).numpy()

Finally we compute the predictions of logistic regression and RF.

In [50]:
from sklearn.linear_model import LogisticRegression

linear = LogisticRegression()
linear.fit(X_train_normalized, y_train)

# We compute the linear preds
linear_pred = linear.predict(X_valid_normalized)

# We compute the random forest predictions
rf_pred = rf.predict(X_valid_normalized)

We can compute all metrics now.

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred, y):
    """Computes all the metrics between predictions and real values"""
    accuracy = accuracy_score(pred,y)
    precision = precision_score(pred,y)
    recall = recall_score(pred,y)
    f1 = f1_score(pred, y)
    return dict(accuracy=accuracy, precision=precision, recall=recall, f1=f1)

models = dict(nrf=nrf_pred, hrf=hrf_pred, rf=rf_pred, linear=linear_pred)

outputs = []
for name, pred in models.items():
    metrics = compute_metrics(pred[:320], y_valid[:320])
    metrics["model"] = name
    outputs.append(metrics)

In [52]:
outputs = pd.DataFrame(outputs)

In [53]:
outputs

,accuracy,precision,recall,f1,model
0,0.809375,0.506494,0.629032,0.561151,nrf
1,0.818750,0.467532,0.679245,0.553846,hrf
2,0.821875,0.415584,0.727273,0.528926,rf
3,0.812500,0.480519,0.649123,0.552239,linear


In [ ]:
outputs.to_csv("results.csv", index = None)

Finally we can have a look at how similar NRF and HRF predictions are.

In [ ]:
(nrf_pred == hrf_pred).mean()

## Exporting featurizer and models

We will now see how we can export the model and the featurizer for the server and the client.

The preprocessing pipeline can be saved for later use on the client side. We can save and load it with pickle.

In [30]:
import pickle

pickle.dump(pipe, open("pipe.pkl", "wb"))
pipe = pickle.load(open("pipe.pkl" , "rb"))

We can also store as well the featurizer, used to shuffle the features before the comparisons. This will be sent to the client side to prepare the data before sending it.

In [31]:
homomorphic_featurizer.save("homomorphic_featurizer.pkl")

Finally, we can store the Homomorphic Random Forest to use on the server side.

In [32]:
pickle.dump(h_rf, open("h_rf.pkl", "wb"))

For the demo built using Streamlit, we need to know what were the values of the categorical features, and also one example of values.

In [33]:
for col in X_train.columns.values:
    if col in categorical_columns:
        X_train[col] = X_train[col].astype("category")
    else:
        X_train[col] = X_train[col].astype(np.float32)

<ipython-input-33-9b2a47c28999>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype(np.float32)
<ipython-input-33-9b2a47c28999>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype("category")


In [34]:
example = X_train.iloc[[0]]
example.to_csv("example.csv", index=None)

In [35]:
dtypes = X_train.dtypes
pickle.dump(dtypes, open("dtypes.pkl", "wb"))

In [36]:
dtypes = pickle.load(open("dtypes.pkl", "rb"))
dtypes

Age               float32
WorkClass        category
fnlwgt            float32
Education        category
EducationNum      float32
MaritalStatus    category
Occupation       category
Relationship     category
Race             category
Gender           category
CapitalGain       float32
CapitalLoss       float32
HoursPerWeek      float32
NativeCountry    category
dtype: object

## Using exported models for inference

In [2]:
import pickle
from cryptotree.cryptotree import HomomorphicTreeFeaturizer, HomomorphicTreeEvaluator

In [3]:
import numpy as np 

dilatation_factor = 16
degree = dilatation_factor

PRECISION_BITS = 28
UPPER_BITS = 9

polynomial_multiplications = int(np.ceil(np.log2(degree))) + 1
n_polynomials = 2
matrix_multiplications = 3

depth = matrix_multiplications + polynomial_multiplications * n_polynomials

poly_modulus_degree = 16384

moduli = [PRECISION_BITS + UPPER_BITS] + (depth) * [PRECISION_BITS] + [PRECISION_BITS + UPPER_BITS]
print(moduli)
print(sum(moduli))

[37, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 37]
438


In [4]:
from cryptotree.seal_helper import create_seal_globals, append_globals_to_builtins
import builtins

create_seal_globals(globals(), poly_modulus_degree, moduli, PRECISION_BITS, use_symmetric_key=False)
append_globals_to_builtins(globals(), builtins)

In [5]:
pipe = pickle.load(open("pipe.pkl","rb"))
homomorphic_featurizer = HomomorphicTreeFeaturizer.load("homomorphic_featurizer.pkl",
                                                        encoder, encryptor, scale)

In [7]:
from cryptotree.tree import SigmoidTreeMaker
from cryptotree.polynomials import polyeval_tree

dilatation_factor = 16
polynomial_degree = dilatation_factor

sigmoid_tree_maker = SigmoidTreeMaker(use_polynomial=True,
                                  dilatation_factor=dilatation_factor, polynomial_degree=polynomial_degree)

h_rf = pickle.load(open("h_rf.pkl","rb"))

tree_evaluator = HomomorphicTreeEvaluator.from_model(h_rf, sigmoid_tree_maker.coeffs, 
                                                   polyeval_tree, evaluator, encoder, relin_keys, galois_keys, 
                                                   scale)

In [9]:
import pandas as pd

x = pd.read_csv("example.csv")
x = pipe.transform(x).reshape(-1)
ctx = homomorphic_featurizer.encrypt(x)

In [10]:
outputs = tree_evaluator(ctx)

In [13]:
ptx = seal.Plaintext()
decryptor.decrypt(outputs, ptx)

homomorphic_pred = encoder.decode_double(ptx)[:2]

print(f"Homomorphic Random Forest output : {homomorphic_pred}")

Homomorphic Random Forest output : [0.7967431717299993, -0.9372202449111074]


In [52]:
ptx = seal.Plaintext()
decryptor.decrypt(outputs, ptx)

homomorphic_pred = encoder.decode_double(ptx)[:2]

pred = rf.predict_proba(x.reshape(1,-1))
neural_pred = model(torch.tensor(x).float().unsqueeze(0))

print(f"Original Random Forest output : {pred}")
print(f"Neural Random Forest output : {neural_pred.detach()}")
print(f"Homomorphic Random Forest output : {homomorphic_pred}")

Original Random Forest output : [[0.714852 0.285148]]
Neural Random Forest output : tensor([[ 0.7526, -0.8541]])
Homomorphic Random Forest output : [0.7528779109157453, -0.9063400792845879]


In [35]:
torch.save(model.state_dict(), "model.pkl")